![](http://blog.codec.wang/cv2_canny_edge_detection_threshold.jpg)

# 目标
- Canny边缘检测的简单概念
- OpenCV函数：cv2.Canny()

# 教程
Canny边缘检测方法常被誉为边缘检测的最优方法，废话不多说，先看个例子：

In [3]:
import cv2
import numpy as np
  
img = cv2.imread('handwriting.jpg', 0)
edges = cv2.Canny(img, 30 ,70)

cv2.imshow('canny', np.hstack((img, edges)))
cv2.waitKey(0)

-1

![](http://blog.codec.wang/cv2_canny_edge_detection.jpg)
cv2.Canny()进行边缘检测，参数2、3表示最低、高阈值，下面来解释下具体原理。

经验之谈：之前我们用低通滤波的方式模糊了图片，那反过来，想得到物体的边缘，就需要用到高通滤波。推荐先阅读：番外篇：图像梯度。

## Canny边缘检测
Canny边缘提取的具体步骤如下：
1. 使用5×5高斯滤波消除噪声：

边缘检测本身属于锐化操作，对噪点比较敏感，所以需要进行平滑处理。高斯滤波的具体内容参考前一篇：平滑图像
```
        ⎡ 1  4  6  4  1 ⎤
        ⎢ 4 16 24 16  4 ⎥
K=1/256 ⎢ 6 24 36 24  6 ⎥
        ⎢ 4 16 24 16  4 ⎥
        ⎣ 1  4  6  4  1 ⎦
```

2，计算图像梯度的方向：

首先使用Sobel算子计算两个方向上的梯度Gx和Gy，然后算出梯度的方向：

θ=arctan(Gy/Gx)

保留这四个方向的梯度：0°/45°/90°/135°，有什么用呢？我们接着看。

3，取局部极大值：

梯度其实已经表示了轮廓，但为了进一步筛选，可以在上面的四个角度方向上再取局部极大值：
![](http://blog.codec.wang/cv2_understand_canny_direction.jpg)
比如，A点在45°方向上大于B/C点，那就保留它，把B/C设置为0。

4，滞后阈值：

经过前面三步，就只剩下0和可能的边缘梯度值了，为了最终确定下来，需要设定高低阈值：
![](http://blog.codec.wang/cv2_understand_canny_max_min_val.jpg)
- 像素点的值大于最高阈值，那肯定是边缘（上图A）
- 同理像素值小于最低阈值，那肯定不是边缘
- 像素值介于两者之间，如果与高于最高阈值的点连接，也算边缘，所以上图中C算，B不算

Canny推荐的高低阈值比在2:1到3:1之间。

## 先阈值分割后检测
其实很多情况下，阈值分割后再检测边缘，效果会更好：

In [4]:
_, thresh = cv2.threshold(img, 0 ,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
edges = cv2.Canny(thresh, 30 ,70)

cv2.imshow('canny', np.hstack((img, thresh, edges)))
cv2.waitKey(0)

-1

代码中我用了番外篇：Otsu阈值法中的自动阈值分割，如果你不太了解，大可以使用传统的方法，不过如果是下面这种图片，推荐用Otsu阈值法。另外Python中某个值不用的话，就写个下划线’_’。

![](http://blog.codec.wang/cv2_canny_edge_detection_threshold.jpg)

In [1]:
import cv2
import numpy as np


def track_back(x):
    pass


img = cv2.imread('sudoku.jpg', 0)
cv2.namedWindow('window')

# 创建滑动条
cv2.createTrackbar('maxVal', 'window', 100, 255, track_back)
cv2.createTrackbar('minVal', 'window', 200, 255, track_back)

while(True):
    # 获取滑动条的值
    max_val = cv2.getTrackbarPos('maxVal', 'window')
    min_val = cv2.getTrackbarPos('minVal', 'window')

    edges = cv2.Canny(img, min_val, max_val)
    cv2.imshow('window', edges)

    # 按下ESC键退出
    if cv2.waitKey(30) == 27:
        break
